## List 1
**Code snippets with `...` are to be filled by you**

In [21]:
import numpy as np
import polars as pl
import scipy

Load the GEFCOM dataset

In [22]:
df = pl.read_csv('data/gefcom.txt', separator='\t', has_header=True)
df

Date,Hour,Price,SystemLoad,Load,Weekday,
f64,f64,f64,f64,f64,f64,str
2.0110101e7,0.0,43.17,15187.0,5091.0,6.0,null
2.0110101e7,1.0,36.24,14464.0,4918.0,6.0,null
2.0110101e7,2.0,34.64,13940.0,4763.0,6.0,null
2.0110101e7,3.0,33.76,13609.0,4660.0,6.0,null
2.0110101e7,4.0,33.08,13391.0,4599.0,6.0,null
…,…,…,…,…,…,…
2.0131217e7,19.0,113.92,23091.0,7167.0,2.0,null
2.0131217e7,20.0,107.26,22504.0,6958.0,2.0,null
2.0131217e7,21.0,89.02,21538.0,6707.0,2.0,null


Recast columns to appropriate types and drop the last column that has empty name ("")

In [23]:
df = df.with_columns(
    pl.col("Hour").cast(pl.UInt8),
    pl.col("Weekday").cast(pl.UInt8),
).drop(pl.col(""))
df

Date,Hour,Price,SystemLoad,Load,Weekday
f64,u8,f64,f64,f64,u8
2.0110101e7,0,43.17,15187.0,5091.0,6
2.0110101e7,1,36.24,14464.0,4918.0,6
2.0110101e7,2,34.64,13940.0,4763.0,6
2.0110101e7,3,33.76,13609.0,4660.0,6
2.0110101e7,4,33.08,13391.0,4599.0,6
…,…,…,…,…,…
2.0131217e7,19,113.92,23091.0,7167.0,2
2.0131217e7,20,107.26,22504.0,6958.0,2
2.0131217e7,21,89.02,21538.0,6707.0,2


### Task 1
Forecast daily average price using:
- naive one-step ahead forecasts,
- simple ETS,
- Holt-Winters (HW).

Use the first 360 days for calibration of the rest for testing.

Create a dataframe with the time series of daily average price

In [24]:
avg = df.group_by("Date").agg(pl.col("Price").mean().alias("Daily Average Price"), pl.col("Weekday").first())
avg = avg.sort("Date")
avg

Date,Daily Average Price,Weekday
f64,f64,u8
2.0110101e7,43.621667,6
2.0110102e7,43.015417,7
2.0110103e7,52.089583,1
2.0110104e7,51.254583,2
2.0110105e7,56.229167,3
…,…,…
2.0131213e7,145.694583,5
2.0131214e7,93.030417,6
2.0131215e7,77.7175,7


Calculate the naive one-step ahead forecasts

In [25]:
...

Ellipsis

Calculate the simple ETS

In [26]:
...

Ellipsis

Implement Holt-Winters:
- fill `...` in the code below
- optimize `params` to minimize MSE on the first 360 days of the data (training set) using `scipy.optimize.minimize`

In [27]:
def holtwinters(params, s, obs):
    """
    Calculates forecasts using Holt-Winters exponential smoothing.

    Args:
        params (tuple): parameters in the form of a 3-tuple (alpha, beta, gamma)
        s (int): seasonality period
        obs (1d numpy array): observed values of the timeseries to forecast
    
    Returns:
        numpy array of forecasts with the length equal to obs.
    """
    alpha, beta, gamma = params
    level = np.full(len(obs), np.nan)
    trend = np.full(len(obs), np.nan)
    season = np.full(len(obs), np.nan)
    forecast = np.full(len(obs), np.nan)

    # set initial values
    level[s-1] = np.mean(obs[:s])
    trend[s-1] = (np.mean(obs[s:2*s]) - np.mean(obs[:s]))/s
    season[:s] = obs[:s] - level[s-1]

    # iteratively compute consecutive forecasts
    for t in range(s, len(obs)-1):
        level[t] = ...
        trend[t] = ...
        season[t] = ...
        forecast[t+1] = ...

    return forecast

Calculate MAE and MSE of naive, ETS and Holt-Winters forecasts on the testing set

In [28]:
...

Ellipsis

### Task 2
Repeat task 1 using the first 720 days for calibration for ETS and HW models. Does the longer calibration window lead to more or less accurate forecasts? Compare the predictions only over the same out-of-sample period, i.e., days #721, #722, ... → why?

In [29]:
...

Ellipsis

### Task 3
Repeat tasks 1 and 2 but forecast all hours of the day, treat prices at each hour of the day as separate time series.

In [30]:
...

Ellipsis